In [7]:
from ultralytics import YOLO, RTDETR
import supervision as sv

import av
import cv2
import yaml
import torch
import time
import json
import numpy as np
import pandas as pd
from pathlib import Path

from icecream import ic

In [12]:
# Load input parameters
source: str = 'D:/Data/Piloto_RC_2024/C2_source_detections.csv'
with open(source, 'r') as csv_file:
    csv_data = pd.read_csv(csv_file, sep=',', names=['frame', 'id', 'class', 'x', 'y', 'w', 'h', 'score'], header=0, index_col=False)
csv_data


,frame,id,class,x,y,w,h,score
0,0,NaN,car,1049.85230,390.644230,123.864746,140.770510,0.938043
1,0,NaN,car,582.66970,294.691200,142.764900,423.462860,0.113602
2,1,NaN,car,1049.76150,390.806900,123.227050,140.381230,0.939039
3,1,NaN,car,588.25500,295.351600,136.499880,422.602260,0.262170
4,2,NaN,car,1049.79250,390.607420,123.080080,140.194820,0.940582
...,...,...,...,...,...,...,...,...
7869,1999,NaN,car,1049.90190,390.373780,123.489500,141.638300,0.939642
7870,1999,NaN,car,548.12270,225.700680,55.856200,70.880490,0.923360
7871,1999,NaN,car,413.92578,252.510900,86.059450,103.513855,0.911740
7872,1999,NaN,car,594.12354,292.531200,131.587160,424.784420,0.668407


In [14]:
result = csv_data.groupby(['frame'])['score'].agg(['count', 'mean']).reset_index()
result

,frame,count,mean
0,0,2,0.525823
1,1,2,0.600604
2,2,2,0.606999
3,3,2,0.622110
4,4,2,0.607321
...,...,...,...
1995,1995,5,0.731684
1996,1996,4,0.848715
1997,1997,5,0.723901
1998,1998,6,0.630804


In [18]:
object_data = result[result['mean'] < 0.4].copy()
object_data

,frame,count,mean
51,51,4,0.337748
247,247,6,0.373047
248,248,6,0.387071
439,439,5,0.386411
440,440,5,0.380721
...,...,...,...
1931,1931,5,0.381898
1956,1956,5,0.376393
1957,1957,5,0.394575
1958,1958,5,0.375167


In [20]:
object_count = object_data[object_data['count'] > 4].copy()
object_count

,frame,count,mean
247,247,6,0.373047
248,248,6,0.387071
439,439,5,0.386411
440,440,5,0.380721
444,444,7,0.352700
445,445,5,0.388211
446,446,5,0.378365
447,447,5,0.399771
453,453,6,0.339080
454,454,6,0.340735


In [23]:
frame_list = object_count['frame'].to_list()
frame_list

[247,
 248,
 439,
 440,
 444,
 445,
 446,
 447,
 453,
 454,
 455,
 456,
 457,
 458,
 459,
 460,
 461,
 496,
 497,
 498,
 499,
 500,
 501,
 502,
 503,
 1539,
 1540,
 1541,
 1542,
 1824,
 1825,
 1826,
 1827,
 1828,
 1829,
 1830,
 1831,
 1832,
 1833,
 1834,
 1835,
 1836,
 1837,
 1839,
 1931,
 1956,
 1957,
 1958,
 1959]

In [ ]:
video_source_path = "D:/Data/Piloto_RC_2024/C2_source.mp4"

container = av.open(video_source_path)
video_stream = container.streams.video[0]



cap = cv2.VideoCapture(video_source_path)
while cap.isOpened():
    


